In [1]:
class Article:
    def __init__(self,article_link, title,doi,issn,journalName,date, abstract,authors,publisher,total_usage='0'):
        self.title=title
        self.article_link=article_link
        self.doi=doi
        self.issn=issn
        self.journal_name=journalName
        self.date_publication=date
        self.abstract=abstract
        self.authors=authors
        self.publisher=publisher
        self.year=date.split(" ")[-1]
        self.total_usage=total_usage
    def to_dict(self):
        return {
            "article_link": self.article_link,
            "title": self.title,
            "doi": self.doi,
            "issn": self.issn,
            "journal_name": self.journal_name,
            "date_publication": self.date_publication,
            "abstract": self.abstract,
            "authors": [{"Name": author.name, "Information": author.information} for author in self.authors],
            "publisher": self.publisher,
            "total_usage": self.total_usage,
            "year": self.year
        }
    def display_article(self):
        print("Title:", self.title)
        print("Article Link:", self.article_link)
        print("DOI:", self.doi)
        print("ISSN:", self.issn)
        print("Journal Name:", self.journal_name)
        print("Publication Date:", self.date_publication)
        print("Abstract:", self.abstract)
        print("Authors:", self.authors)
        print("Publisher:", self.publisher)
        print("Year:", self.year)
        print("Total Usage:", self.total_usage)
        

In [2]:
class Author:
    def __init__(self, name, information):
        self.name = name
        self.information = information
    def to_dict(self):
        return {
            'Name': self.name,
            'Information': self.information
        }
auth=Author("hey","uu")
a=Article("a","a","a","a","a","a","a",[auth],"a")
a.to_dict()

{'article_link': 'a',
 'title': 'a',
 'doi': 'a',
 'issn': 'a',
 'journal_name': 'a',
 'date_publication': 'a',
 'abstract': 'a',
 'authors': [{'Name': 'hey', 'Information': 'uu'}],
 'publisher': 'a',
 'total_usage': '0',
 'year': 'a'}

In [3]:
def scrapeArticle(driver2,url):
    o=0
    #Articles scraper :
    driver2.get(url)
    wait = WebDriverWait(driver2, 10)
    try:
        wait.until(EC.visibility_of_element_located((By.ID,"CybotCookiebotDialogBodyLevelButtonLevelOptinAllowallSelectionWrapper"))).click()
    except:
        o=o+1

        #"loa__link.w-slide__btn.tab-link.slide-active"
    time.sleep(2)

    title=driver2.find_element(By.CLASS_NAME,"citation__title").text
    abstract=driver2.find_element(By.XPATH,"//*[@id='skip-to-main-content']/main/div[1]/article/div[2]/div[2]/div[2]/div[1]/div/div[2]/div/p").text
#     print(abstract)
    try:
        while  EC.visibility_of_element_located((By.XPATH, "//a[@id='pill-information__contentcon']")):
            driver2.find_element(By.XPATH,"//*[@id='skip-to-main-content']/main/div[1]/article/div[2]/div[3]/div/ul/li[1]/button").click()
    except:
        o=o+1
    journalInfo=driver2.find_elements(By.CLASS_NAME,"cover-image__details")[0]
    journalTitle=journalInfo.find_element(By.CLASS_NAME,"serial-title").text
    date_de_publication=journalInfo.find_element(By.CLASS_NAME,"cover-date").text
    ISSN=journalInfo.find_element(By.CLASS_NAME,"cover-image__details-extra").find_elements(By.CLASS_NAME,"space")[0].text
    DOI=journalInfo.find_element(By.CLASS_NAME,"cover-image__details-extra").find_elements(By.CLASS_NAME,"space")[2].text
    tagsElement=driver2.find_element(By.CLASS_NAME,"tags-widget__content")
    publisher=driver2.find_element(By.XPATH,"//*[@id='pill-information__content']/div/div[4]/div").find_element(By.CLASS_NAME,"publisher__name").text
#     print("publisher : "+publisher)
    
    keywords=[]
    for element in tagsElement.find_elements(By.CLASS_NAME,"badge-type"):
#         print(element.text+" my text")
        keywords.append(element.text)
#     print(len(tagsElement.find_elements(By.CLASS_NAME,"badge-type")))

#     print("DOI : "+DOI)
#     print("JournalTitle : "+journalTitle)
#     print("date de publication : "+date_de_publication)
#     print("ISSN : "+ISSN)
#     for keyword in keywords:
#         print(keyword)
    #     break
    try:
        while  EC.visibility_of_element_located((By.XPATH, "//a[@id='pill-information__contentcon']")):
            driver2.find_element(By.XPATH,"//*[@id='pill-authors__contentcon']").click()
            break
    except:
        o=o+1
    authors=[]
    for auth in driver2.find_element(By.CLASS_NAME,"pill-authors__content").find_elements(By.CLASS_NAME,"auth-info"):
        name=auth.find_element(By.CLASS_NAME,"auth-name").text
        info=auth.find_element(By.CLASS_NAME,"info--text.auth-institution").text
#         print("Name : "+name)
#         print("info : "+info)
        authors.append(Author(name, info))

#     print(len(driver2.find_elements(By.CLASS_NAME,"auth-info")))
#     def __init__(self,article_link, title,doi,issn,journalName,date, abstract,authors,publisher,total_usage):
    return Article(url,title,DOI,ISSN,journalTitle,date_de_publication,abstract,authors,publisher)
    
#     

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrapeTotalUsage(driver2,url):
    print("the url i received "+ url)
    driver2.get(url)
    return driver2.find_element(By.XPATH,"//*[@id='pill-bibliometrics__content']/div/div[1]/div/ul[1]/li[2]/div/div/span").text

    

In [6]:
import json
import csv

# Load JSON data
with open('articles.json', 'r') as json_file:
    json_data = json.load(json_file)

# Extract URLs from JSON data
json_urls = set(article['article_link'] for article in json_data)

# Load URLs from CSV file
csv_urls = set()
with open('myurls.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        csv_urls.add(row[0])  # Assuming URLs are in the first column of the CSV file

# Find URLs that are in CSV but not in JSON
unique_urls = csv_urls - json_urls

# Print or process unique URLs
print(len(json_data))


227


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
chromeoptions = Options()
chromeoptions.add_argument('--headless')
driver2=webdriver.Chrome(options=chromeoptions)
driver3=webdriver.Chrome(options=chromeoptions)
# print(len(failedurls))
failedurls=[]
scrapedSArticles=[]
for url in unique_urls:
    try:
        article=scrapeArticle(driver2,url)
        article.total_usage=scrapeTotalUsage(driver2,url)
        scrapedSArticles.append(article)
        
    except:
        failedurls.append(url)

the url i received https://dl.acm.org/doi/10.1145/3417516
the url i received https://dl.acm.org/doi/10.1145/3531016
the url i received https://dl.acm.org/doi/10.1145/3603714
the url i received https://dl.acm.org/doi/10.1145/3408310
the url i received https://dl.acm.org/doi/10.1145/3341104
the url i received https://dl.acm.org/doi/10.1145/3417987
the url i received https://dl.acm.org/doi/10.1145/3595179
the url i received https://dl.acm.org/doi/10.1145/3199665
the url i received https://dl.acm.org/doi/10.1145/3315571
the url i received https://dl.acm.org/doi/10.1145/3511904
the url i received https://dl.acm.org/doi/10.1145/3448649
the url i received https://dl.acm.org/doi/10.1145/3511903
the url i received https://dl.acm.org/doi/10.1145/3483332
the url i received https://dl.acm.org/doi/10.1145/3522582
the url i received https://dl.acm.org/doi/10.1145/3397202
the url i received https://dl.acm.org/doi/10.1145/3432250
the url i received https://dl.acm.org/doi/10.1145/3529097
the url i rece